In [304]:
import numpy as np
import pandas as pd
import urllib
import urllib.request as req
from bs4 import BeautifulSoup as bs
from datetime import datetime

In [289]:
now = datetime.now()    # 현재 시간

# url query 검색주기를 1일로 활용할 것이므로
ds = "{0}.{1}.{2}.{3}.{4}".format(now.year, now.month, now.day, now.hour, now.minute)   #검색일
de = "{0}.{1}.{2}.{3}.{4}".format(now.year, now.month, now.day-1, now.hour, now.minute)   #전일

In [290]:
# datetime를 활용해서 구한 ds, de (어제,오늘 날짜)를 f_formatting으로 url에 넣어줌
url = "https://search.naver.com/search.naver?&where=news&query=춘천&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=4&ds={0}&de={1}&docid=&nso=so:r,p:1d,a:all&mynews=0&cluster_rank=38&start=1&refresh_start=0".format(ds, de)
## url query 검색기간 옵션   pd=4: 당일, pd=1: 1주, pd=2: 1개월

raw = requests.get(url)
html = raw.content
soup = bs(html, 'html.parser')

In [291]:
# html 참고
links = soup.select('ul.type01>li>dl>dd.txt_inline>a')
links

[<a class="_sp_each_url" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=102&amp;oid=001&amp;aid=0011703148" onclick="return goOtherCR(this, 'a=nws*a.nav&amp;r=39&amp;i=880000D8_000000000000000011703148&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a>,
 <a class="_sp_each_url" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=102&amp;oid=003&amp;aid=0009933648" onclick="return goOtherCR(this, 'a=nws*a.nav&amp;r=44&amp;i=88000127_000000000000000009933648&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a>,
 <a class="_sp_each_url" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=102&amp;oid=421&amp;aid=0004717835" onclick="return goOtherCR(this, 'a=nws*a.nav&amp;r=48&amp;i=08138263_000000000000000004717835&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a>,
 <a class="_sp_each_url" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=102&amp;oid=056&amp;aid=0010858531" oncli

In [292]:
a_tags = soup.findAll('a',{'class':'_sp_each_url'})
h_list = []
for n in range(1, 150, 10):
    raw_url = requests.get("https://search.naver.com/search.naver?&where=news&query=춘천&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=4&ds={0}&de={1}&docid=&nso=so:r,p:1d,a:all&mynews=0&cluster_rank=38&start=1"+str(n)+"&refresh_start=0".format(ds, de))
    html = raw_url.content
    soup = bs(html, 'html.parser')
    for a_tag in a_tags:
        
        # 사이트별 html code가 상이하므로 네이버 뉴스기사만 크롤링해서 h_list에 주소 추가
        if 'naver' in a_tag['href'] :
            h_list.append(a_tag['href'])

In [344]:
article_body = []
for i in range(len(h_list)):
    each_url = requests.get(h_list[i])
    each_html = bs(each_url.text, 'html.parser')
    
    content = each_html.select_one('#articleBodyContents') # content변수에 articleBodyContents 내용 저장
    output = ''
    for item in content.contents:
        stripped = str(item).strip()
        if stripped == '':
            continue
        if stripped[0] not in ['<','/']:
            output += str(stripped)
            output = output.replace('본문 내용TV플레이어','').strip()
    article_body.append(output)

In [345]:
article_body

['(춘천=연합뉴스) 이상학 기자 = 강원 춘천지역에서 올해 처음으로 중증열성혈소판감소증후군(SFTS) 환자가 발생했다.춘천시는 지난 22일 발열과 근육통 증상으로 병원을 찾아 입원 치료를 받던 50대 여성 A씨가 SFTS 검사결과 양성 판정을 받았다고 25일 밝혔다.이로써 강원도 내 SFTS 감염 환자는 올해 들어 원주 2명, 화천 1명에 이어 모두 4명으로 늘었다.춘천시보건소는 최근 기온이 올라가면서 진드기 활동이 활발해지고 농사일이나 등산 등 야외생활 증가에 따라 SFTS 감염에 대한 주의가 필요하다고 강조했다.SFTS에 감염되면 발열과 근육통, 설사, 식욕부진, 구역질, 두통이 나타나는 것으로 알려져 있다.시보건소는 야외활동 시 긴 옷을 입고 활동 후 목욕, 입었던 옷을 세탁해야 한다고 당부했다.춘천시는 예방을 위해 읍·면 지역 주민을 대상으로 진드기 기피제 4천700개를 배부했다.또 현재 산책로 등 80곳에 설치된 태양광 해충 기피제 자동분사기를 이달 중 10곳에 추가 설치할 방침이다.춘천시 관계자는 "야생진드기 매개 감염병은 5월부터 8월까지 전국에서 발생한다"며 "야외활동 후 2주 이내에 고열과 구토, 설사, 근육통 등의 증상이 있으면 반드시 의료기관을 방문해 치료해야 한다"고 조언했다.hak@yna.co.kr',
 '【춘천=서정욱 기자】춘천에서 올해 처음으로 중증 열성 혈소판 감소 증후군(SFTS) 환자가 발생했다.25일 춘천시에 따르면 지난 22일 발열과 근육통 증상으로 병원을 방문해 입원 치료를 받고 있던 50대 여성 A씨가 SFTS 검사를 한 결과 양성 판정을 받았다 고 밝혔다.역학조사 결과, A씨는 지난 5월에서 6월 한 달간 경북 구미시 친정집에 방문했으며 기르던 강아지의 진드기에 물린 후, 감염된 것으로 파악하고 있다.한편, 춘천에서 첫 SFTS 환자 발생에 따라 올해 강원도 내 SFTS 감염 환자는 화천 1명, 횡성1명, 원주 2명, 춘천 1명등 총 5명으로 늘었다.춘천시 보건소 관계자는 “최근 기온이 상승하면서 진드기 활동이 활발해